In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import patches
from mplex import Grid

In [2]:
out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)
fmt = ".pdf"
palette = list(sns.color_palette("Set2"))
data = [
    (1, 1, 0, 0, 0, 0, 0, 0, 10),
    (0, 1, 1, 0, 0, 0, 0, 0, 10),
    (1, 1, 1, 0, 3, 1 + 1j, 4, 5, 11),
    (1, 0, 1, 0, 0, 5 + 1j, 2, 3, 10),
]

df = pd.DataFrame(
    data,
    columns=[
        "KalTA4u508",
        "lhx9:Gal4",
        "UAS:ChR2-mCherry",
        "No 2P, no opto.",
        "2P only",
        "Opto. + 2P",
        "Opto. only",
        "Tested",
        "Responded fish",
    ],
)
df = df.iloc[:, [2, 0, 1, 3, 4, 6, 5, 7, 8]]
df = df.iloc[[0, 1, 3, 2]]

In [ ]:
g = Grid((20, 60), (1, 4), spines="lb")
g.set_visible_sides(ticks="l")
g[:, 1:].set_visible_sides(spines="b", ticks=False)
for i, row in enumerate(df.iloc[:, 3:7].values):
    ax = g[0, i]
    ax.bar(
        np.arange(4),
        np.real(row),
        color=palette[:4],
        width=1,
        ec="k",
        label=df.columns[3:7],
        lw=0.5,
    )
    ax.bar(
        np.arange(4),
        np.imag(row),
        bottom=np.real(row),
        color=palette[:4],
        width=1,
        ec="k",
        lw=0.5,
        hatch="/" * 9,
        clip_on=False,
    )
    ax.spines["bottom"].set_bounds(-0.5, 3.5)
    ax.set_xlim(-1, 4)

kw = dict(textcoords="offset points", va="top", fontsize=7)
for i, row in enumerate(df.iloc[:, :3].values.T):
    ax = g[0, 0]
    pad = (0, -9 * i - 6)
    ax.annotate(
        df.columns[i],
        (-1, 0),
        pad,
        ha="right",
        style="italic",
        **kw,
    )
    for j, thing in enumerate(row):
        ax = g[0, j]
        ax.annotate("−+"[thing], (1.5, 0), pad, ha="center", **kw)

ax = g[0, 0]
pad = (0, -9 * 3 - 6 - 3)

ax.annotate(
    "Responded/tested fish",
    (-1, 0),
    pad,
    ha="right",
    **kw,
)

for j, thing in enumerate(df.values[:, -2:]):
    ax = g[0, j]
    ax.annotate(f"{int(thing[0])}/{int(thing[1])}", (1.5, 0), pad, ha="center", **kw)

ax = g[0, 0]
ax.set_ylim([0, 6])
ax.set_ylabel("# of prey\ncapture events")
ax = g[0, -1]
handles, labels = ax.get_legend_handles_labels()
handles.append(
    patches.Rectangle((0, 0), 1, 1, hatch="/" * 9, ec="k", fc="none", lw=0.5)
)
labels.append("Outside of opto.\nstimulation period")
ax.legend(
    handles=handles,
    labels=labels,
    frameon=False,
    bbox_to_anchor=(1.2, 1),
    loc="upper left",
    borderpad=0,
    borderaxespad=0,
    handletextpad=0.4,
    handlelength=1.25,
    fontsize=7,
    handleheight=0.5,
)
if fmt:
    g.savefig((out_dir / "control").with_suffix(fmt))